In [1]:
import os
import urllib
import fastai.vision.all as fai_vision
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import platform
import pathlib
import altair as alt
import pandas as pd

In [2]:
EXTERNAL_DEPENDENCIES = {
    "models/fish_mask_model_2021_08_17.pth": {
        "url": "https://www.dropbox.com/s/e9c4oi6tf5qnqyd/fish_mask_model_2021_08_17.pth?dl=1",
        "size": 494929527
    },
    "models/perumixed3.pkl": {
        "url": "https://www.dropbox.com/s/31e6wuwrlm66sco/perumixed3.pkl?dl=1",
        "size": 179319095
    }
}

In [3]:
def download_file(file_path):
    # Don't download the file twice. (If possible, verify the download using the file length.)
    if os.path.exists(file_path):
        if "size" not in EXTERNAL_DEPENDENCIES[file_path]:
            return
        elif os.path.getsize(file_path) == EXTERNAL_DEPENDENCIES[file_path]["size"]:
            return
    try:
        with open(file_path, "wb") as output_file:
            with urllib.request.urlopen(EXTERNAL_DEPENDENCIES[file_path]["url"]) as response:
                length = int(response.info()["Content-Length"])
                counter = 0.0
                MEGABYTES = 2.0 ** 20.0
                while True:
                    data = response.read(8192)
                    if not data:
                        break
                    counter += len(data)
                    output_file.write(data)
    except:
        pass
    return

In [4]:
for filename in EXTERNAL_DEPENDENCIES.keys():
    download_file(filename)

In [22]:
def load_unet_model():
    data_loader = fai_vision.SegmentationDataLoaders.from_label_func(
        path = Path("."),
        fnames = [Path('thumbnails/DSC_0001-CV1-TRAMPA.JPG') for i in range(4)],
        label_func = lambda x: x,
        codes = np.array(["Photo", "Masks"], dtype=str),
        item_tfms = [fai_vision.Resize(256, method = 'squish'),],
        batch_tfms = [fai_vision.IntToFloatTensor(div_mask = 255)],
        bs=4,
        valid_pct = 0.2, num_workers = 0)
    segmenter = fai_vision.unet_learner(data_loader, fai_vision.resnet34)
    segmenter.load('fish_mask_model_2021_08_17')
    return data_loader, segmenter

In [23]:
unet_data_loader, segmenter = load_unet_model()

In [24]:
def mask_fish_pil(unmasked_fish, fastai_mask):
    unmasked_np = np.array(unmasked_fish)
    np_mask = fastai_mask.argmax(dim=0).numpy()
    total_pixels = np_mask.size
    fish_pixels = np.count_nonzero(np_mask)
    percentage_fish = (fish_pixels / total_pixels) * 100
    np_mask = (255 / np_mask.max() * (np_mask - np_mask.min())).astype(np.uint8)
    np_mask = np.array(Image.fromarray(np_mask).resize(unmasked_np.shape[1::-1], Image.BILINEAR))
    np_mask = np_mask.reshape(*np_mask.shape, 1) / 255
    masked_fish_np = (unmasked_np * np_mask).astype(np.uint8)
    masked_fish_pil = Image.fromarray(masked_fish_np)
    return masked_fish_pil, percentage_fish

In [26]:
fish_files = Path('thumbnails').glob('*.JPG')
len(list(fish_files))

1418

In [33]:
test_files = list(Path('thumbnails').glob('*.JPG'))
input_dl = segmenter.dls.test_dl(test_files)
masks, _ = segmenter.get_preds(dl=input_dl)
len(masks)

1418

In [34]:
mask_result = []
mask_dir = Path('masked')
for idx, test_file in enumerate(test_files):
    original_pil = Image.open(test_file)
    masked_pil, percentage_fish = mask_fish_pil(original_pil, masks[idx])
    mask_result.append({'file_name':test_file.name,
                        'mask_percent':percentage_fish})
    mask_file = mask_dir / test_file.name
    masked_pil.save(mask_file)

In [35]:
mask_df = pd.DataFrame(mask_result)
mask_df.head()

,file_name,mask_percent
0,DSC_0385.JPG,10.467529
1,DSC_0095-CHT1.JPG,13.569641
2,DSC_0391.JPG,20.661926
3,DSC_1058.JPG,5.815125
4,DSC_0346.JPG,26.457214


In [38]:
mask_df.to_csv('mask_results.tsv', sep='\t', index=False)